In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

def fullDisplay():
    pd.set_option("display.max_rows", None, "display.max_columns", None)

def defaultDisplay():
    pd.reset_option('^display.', silent=True)

In [2]:
welfare = pd.read_csv("Data/welfare_clean.csv", low_memory=False)
treatments = welfare['w']
labels = welfare['y']
welfare.drop(columns=['w', 'y'], inplace=True)
welfare

,year,id,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,...,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss,attblack,attblack_miss
0,0,1,7,0.004845,0.005228,1,135,0.005641,2,2,...,0,0,0,0,0,0,0,0,0.005440,0
1,0,2,1,0.005055,0.005228,0,106,0.006538,2,2,...,0,1,0,0,1,0,0,0,0.004080,0
2,0,3,7,0.004845,0.005228,1,99,0.006538,2,2,...,0,1,0,0,0,0,0,0,0.002040,0
3,0,4,3,0.005055,0.005228,0,142,0.004615,2,0,...,0,0,0,0,1,0,0,0,0.004080,0
4,0,5,8,0.005055,0.005228,1,211,0.005171,2,1,...,0,0,0,0,0,0,0,0,0.004080,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,15,2040,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.004080,0
36497,15,2041,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.006120,0
36498,15,2042,7,0.004845,0.005228,1,211,0.005171,2,2,...,0,1,0,0,0,1,0,0,0.004080,0
36499,15,2043,7,0.005935,0.005228,1,211,0.005171,2,2,...,0,1,0,0,1,1,0,0,0.005021,1


In [3]:
N = 36501

cov = welfare.cov()
means = welfare.mean(axis=0)

X = np.random.multivariate_normal(means.values, cov, size=1000, check_valid='warn', tol=1e-8)

poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
poly.fit(X[:, importantFeatureIndices[:3]])
higherNames = poly.get_feature_names(input_features=importantFeatureNames[:3])
full = poly.transform(X[:, importantFeatureIndices[:3]])
higher = full[:, -(full.shape[1] - 3):] # select only higher order

treatment_probability = 0.5
treat_elements = [0, 1]
treat_probabilities = [1 - treatment_probability, treatment_probability]

# randomly assigned treatments with propensity treatment_probability
treatments = np.random.choice(treat_elements, size=N, p=treat_probabilities).reshape((-1, 1))

NameError: name 'importantFeatureIndices' is not defined

In [189]:
def dgp(welfare, effect_type="heterogeneous", effect_homogeneous=10, effect_heterogeneous=2,
        treatment_type="binary", treatment_probability=0.5, heterogeneous_select=4, order=3, linearity="med", N=1000):    
    
    featureNames = list(welfare.columns)

    importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
    importantFeatureIndices = []
    for name in importantFeatureNames:
        importantFeatureIndices.append(featureNames.index(name)) 

    # error terms
    error = np.random.normal(size=(N,1))

    # data generation
    cov = welfare.cov()
    means = welfare.mean(axis=0)
    X = np.random.multivariate_normal(means.values, cov, size=1000, check_valid='warn', tol=1e-8)

    # linearity specification
    if linearity != 'full':
        select = 0 # select n most important features for interactions and polynomials
        poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

        if linearity == "high": 
            select = 2
        elif linearity == "med": 
            select = 4
        elif linearity == "low": 
            select = 8
        else: # if some typo, assume baseline of high
            select = 2

        poly.fit(X[:, importantFeatureIndices[:select]])
        fullData = poly.transform(X[:, importantFeatureIndices[:select]])
        fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
        higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
        higherNames = fullNames[-(len(fullNames) - select):]

        X = np.append(X, higherData, axis=1) 
        featureNames.extend(list(higherNames))

    # Treatment selection
    if treatment_type == "binary":
        # randomly assigned treatments with propensity treatment_probability
        treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
        treated = treatments > 0

        # generate counterfactual treatment vector
        c_treatments = deepcopy(treatments)
        c_treatments[treatments == 0] = 1
        c_treatments[treatments == 1] = 0
    else:
        treatments = np.random.uniform(size=(N, 1))
        treated = treatments > 0.5

        # to-do: counterfactual treatment vector
        c_treatments = deepcopy(treatments)

    # Treatment effect calculation
    if effect_type == "homogeneous":

        T = treatments*effect_homogeneous
        CT = c_treatments*effect_homogeneous

    else: 
        # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
        heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]

        T = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
        T = T.reshape(-1, 1)
        T[~treated] = np.zeros((~treated).sum())
        
        # counterfactual effect
        CT = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
        CT = CT.reshape(-1, 1)
        CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual

    # Outcome calculation
    betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
    y = T + X@betas + error
    cy = CT + X@betas + error
    
    # ATE calculation
    empirical_treated = y[treated] - cy[treated] # the treated (in y)
    empirical_untreated = cy[~treated] - y[~treated] # the untreated
    ate = (empirical_treated.mean() + empirical_untreated.mean())/2
    
    return y, X, betas, featureNames, treatments, cy, ate

Below is to implement / have running state of variables

In [170]:
np.random.seed(123)
effect_type="heterogeneous"
effect_homogeneous=10
effect_heterogeneous=2
# treatment_type="continuous"
treatment_type="binary"
treatment_probability=0.5
heterogeneous_select=4
order=3
linearity="med"
N=1000

featureNames = list(welfare.columns)

importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
importantFeatureIndices = []
for name in importantFeatureNames:
    importantFeatureIndices.append(featureNames.index(name)) 

# error terms
error = np.random.normal(size=(N,1))

# data generation
cov = welfare.cov()
means = welfare.mean(axis=0)
X = np.random.multivariate_normal(means.values, cov, size=1000, check_valid='warn', tol=1e-8)

# linearity specification
if linearity != 'full':
    select = 0 # select n most important features for interactions and polynomials
    poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

    if linearity == "high": 
        select = 2
    elif linearity == "med": 
        select = 4
    elif linearity == "low": 
        select = 8
    else: # if some typo, assume baseline of high
        select = 2

    poly.fit(X[:, importantFeatureIndices[:select]])
    fullData = poly.transform(X[:, importantFeatureIndices[:select]])
    fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
    higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
    higherNames = fullNames[-(len(fullNames) - select):]

    X = np.append(X, higherData, axis=1) 
    featureNames.extend(list(higherNames))

# treatment type
if treatment_type == "binary":
    # randomly assigned treatments with propensity treatment_probability
    treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
    treated = treatments > 0
    
    # generate counterfactual treatment vector
    c_treatments = deepcopy(treatments)
    c_treatments[treatments == 0] = 1
    c_treatments[treatments == 1] = 0

else:
    treatments = np.random.uniform(size=(N, 1))
    treated = treatments > 0.5
    c_treatments = deepcopy(treatments)
    # to-do
    
# treatment effect
if effect_type == "homogeneous":

    T = treatments*effect_homogeneous
    CT = c_treatments*effect_homogeneous
    
else: # heterogeneous
    # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
    heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]
    
    T = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
    T = T.reshape(-1, 1)
    T[~treated] = np.zeros((~treated).sum())
    
    CT = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
    CT = CT.reshape(-1, 1)
    CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual
    

betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
y = T + X@betas + error
cy = CT + X@betas + error

In [184]:
X[:, heterogeneousIndices].mean(axis=0).sum() # average across the used columns to check ATE

15.720148792484203

In [176]:
tmp = y[treated] - cy[treated] # the treated
untmp = cy[~treated] - y[~treated] # the untreated
ate = (tmp.mean() + untmp.mean())/2

32.43845628547201

In [187]:
y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, N=1000, order=3, treatment_probability=0.5)